In [3]:

import sys
sys.executable

'/bin/python3'

In [4]:
import pyspark
import os
from pyspark.sql import SparkSession
from pyspark import SparkContext
from  pyspark.sql.functions import input_file_name
from pyspark.sql import functions as F
from pyspark.sql import Row
from pyspark.sql.functions import row_number,lit
from pyspark.sql.window import Window

In [22]:
def flatten_df(nested_df):
    flat_cols = [c[0] for c in nested_df.dtypes if c[1][:6] != 'struct']
    nested_cols = [c[0] for c in nested_df.dtypes if c[1][:6] == 'struct']

    flat_df = nested_df.select(flat_cols +
                               [F.col(nc+'.'+c).alias(nc+'_'+c)
                                for nc in nested_cols
                                for c in nested_df.select(nc+'.*').columns])
    return flat_df

In [7]:
spark = SparkSession.builder \
    .master("local[*]") \
    .appName('test') \
    .getOrCreate()

22/09/23 18:07:38 WARN Utils: Your hostname, vyago-desktop resolves to a loopback address: 127.0.1.1; using 192.168.18.7 instead (on interface enp4s0)
22/09/23 18:07:38 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
22/09/23 18:07:38 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [8]:
spark

In [9]:
reviews_dir = "/home/vyago/Documents/steam-data-engineering/reviews/"
gameid_dir = "221380"
file = "416F4A347850666B6C4F6F43633975777677453D.json"

## ONE SAMPLE FILE

In [ ]:
test = os.path.join(os.path.join(reviews_dir, gameid_dir), file)
test

In [ ]:
raw_df = spark.read.json(test)
raw_df.printSchema()
raw_df.show()

In [ ]:
rows = raw_df.select("reviews").take(1)
df = spark.createDataFrame(rows[0][0])
df_flatten = flatten_df(df)
df_flatten.take(1)

In [ ]:
df_flatten.columns

# ALL FILES FROM FOLDER

In [ ]:
test_dir = os.path.join(reviews_dir, gameid_dir)
test_dir

In [ ]:
all_dfs = spark.read.json(test_dir)
all_dfs.printSchema()
all_dfs.show()

In [ ]:
rdd = all_dfs.rdd

In [ ]:
rdd.count()

In [ ]:
def proc_json(raw_df)  :
    rows = raw_df["reviews"]
    return rows


rdd2 = rdd.flatMap(proc_json) \
          .collect()

df = spark.createDataFrame(rdd2)
df = flatten_df(df)
df = df.withColumn('filename', input_file_name()) # dont work
df.printSchema()

In [ ]:

df.take(1)

In [ ]:
df.select("author_steamid").show()

# ALL FILES FROM ALL FOLDERS

- https://stackoverflow.com/questions/32233575/read-all-files-in-a-nested-folder-in-spark

In [18]:
reviews_lite_dir = "/home/vyago/Documents/steam-data-engineering/reviews_lite/*/*"

Df schema of all files of all dirs : 

In [19]:
all_dfs = spark.read.json(reviews_lite_dir).withColumn("filename", input_file_name())
all_dfs.printSchema()
all_dfs.show()

root
 |-- cursor: string (nullable = true)
 |-- query_summary: struct (nullable = true)
 |    |-- num_reviews: long (nullable = true)
 |    |-- review_score: long (nullable = true)
 |    |-- review_score_desc: string (nullable = true)
 |    |-- total_negative: long (nullable = true)
 |    |-- total_positive: long (nullable = true)
 |    |-- total_reviews: long (nullable = true)
 |-- reviews: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- author: struct (nullable = true)
 |    |    |    |-- last_played: long (nullable = true)
 |    |    |    |-- num_games_owned: long (nullable = true)
 |    |    |    |-- num_reviews: long (nullable = true)
 |    |    |    |-- playtime_at_review: long (nullable = true)
 |    |    |    |-- playtime_forever: long (nullable = true)
 |    |    |    |-- playtime_last_two_weeks: long (nullable = true)
 |    |    |    |-- steamid: string (nullable = true)
 |    |    |-- comment_count: long (nullable = true)
 |    |    |--

In [34]:
rdd = all_dfs.rdd
rdd.count()

97

Obtaining reviews :

In [25]:
def proc_json(raw_df, field)  :
    rows = raw_df[field]
    return(rows)

rdd_reviews = rdd.flatMap(lambda item : proc_json(item, field = "reviews") )
df_reviews  = spark.createDataFrame(rdd_reviews)
df_reviews.printSchema()

root
 |-- author: struct (nullable = true)
 |    |-- last_played: long (nullable = true)
 |    |-- num_games_owned: long (nullable = true)
 |    |-- num_reviews: long (nullable = true)
 |    |-- playtime_at_review: long (nullable = true)
 |    |-- playtime_forever: long (nullable = true)
 |    |-- playtime_last_two_weeks: long (nullable = true)
 |    |-- steamid: string (nullable = true)
 |-- comment_count: long (nullable = true)
 |-- language: string (nullable = true)
 |-- received_for_free: boolean (nullable = true)
 |-- recommendationid: string (nullable = true)
 |-- review: string (nullable = true)
 |-- steam_purchase: boolean (nullable = true)
 |-- timestamp_created: long (nullable = true)
 |-- timestamp_updated: long (nullable = true)
 |-- voted_up: boolean (nullable = true)
 |-- votes_funny: long (nullable = true)
 |-- votes_up: long (nullable = true)
 |-- weighted_vote_score: string (nullable = true)
 |-- written_during_early_access: boolean (nullable = true)



Getting corresponding filename from each game

In [26]:
rdd_filenames = rdd.map(  lambda item : [ proc_json(item, field = "reviews"),
                                          proc_json(item, field = "filename")] ) \
                   .flatMap(lambda item:  [item[1] for i in item[0] ]) \
                   .map(lambda item : Row(id  = item, ))

df_filenames  = spark.createDataFrame(rdd_filenames)
df_filenames.printSchema()

root
 |-- id: string (nullable = true)



Checking lenght of both dfs are equal

In [27]:

df_reviews.count() == df_filenames.count()

True

In [28]:
df_reviews.count()

9300

Join dataframes

In [29]:
w = Window().orderBy(lit('A'))
df_reviews = df_reviews.withColumn("row_num", row_number().over(w))
df_filenames = df_filenames.withColumn("row_num", row_number().over(w))

df_reviews = df_reviews.join(df_filenames,df_reviews.row_num ==  df_filenames.row_num,"inner")

In [30]:

df_reviews.select("id").show()

22/09/23 18:10:06 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
22/09/23 18:10:06 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.


+--------------------+
|                  id|
+--------------------+
|file:/home/vyago/...|
|file:/home/vyago/...|
|file:/home/vyago/...|
|file:/home/vyago/...|
|file:/home/vyago/...|
|file:/home/vyago/...|
|file:/home/vyago/...|
|file:/home/vyago/...|
|file:/home/vyago/...|
|file:/home/vyago/...|
|file:/home/vyago/...|
|file:/home/vyago/...|
|file:/home/vyago/...|
|file:/home/vyago/...|
|file:/home/vyago/...|
|file:/home/vyago/...|
|file:/home/vyago/...|
|file:/home/vyago/...|
|file:/home/vyago/...|
|file:/home/vyago/...|
+--------------------+
only showing top 20 rows



In [31]:
df_reviews.count()

22/09/23 18:10:29 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
22/09/23 18:10:29 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.


9300